In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Import static data

In [ ]:
df1 = pd.read_json("./data/w01-18.json")
df2 = pd.read_json("./data/w19-34.json")
df3 = pd.read_json("./data/w35-49.json")
df4 = pd.read_json("./data/w50-52.json")

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

# Load data from web

### Utilities functions to handle date & time

In [ ]:
def getWeekNumber(date):
    return date.isocalendar()[1]

def getYear(date):
    return date.isocalendar()[0]

def getWeekday(date):
    return date.isoweekday()

def getStrDate(date):
    return date.strftime("%Y-%m-%d")

def getYearAndWeek(date):
    y = str(getYear(date)) + (str(getWeekNumber(date)) if getWeekNumber(date)>10 \
        else '0' + str(getWeekNumber(date)))
    return y

In [ ]:
def keepFullDays(df):
    fullDayMask = df.groupby(lambda x: getStrDate(df.loc[x]["ts"])).count() == 24
    nonFullDays = fullDayMask[fullDayMask["open"] == False].index
    hourTickerInNonFullDaysMask = df["ts"].apply(getStrDate).isin(nonFullDays)
    res_df = df[~hourTickerInNonFullDaysMask]
    return res_df

In [ ]:
# Load data and save it to file
import urllib.request

nb_days = 30
now = datetime.now()
now.isoformat()
now_str = now.strftime("%Y-%m-%d")

start = now - timedelta(days=nb_days)
start_str = start.strftime("%Y-%m-%d")

filename = "./data/" + start_str +"-"+now_str+".json"

url = "https://bitcoincharts.com/charts/chart.json?m=bitstampUSD&r={nb_days}&i=Hourly".format(nb_days=str(nb_days))
response = urllib.request.urlopen(url)
data = response.read()

f = open(filename, 'wb')
f.write(data)
f.close()


In [ ]:
def formatDataFrameFromJSON(filename):
    df = pd.read_json(filename)
    
    # Set columns and index
    df.columns = ["ts", "open", "high", "low", "close", "vol_btc", "vol_cur", "weighted_price"]
    df["ts"] = pd.to_datetime(df["ts"], unit="s")
    df = df.set_index("ts")
    # reset index b/c it was removed
    df["ts"] = df.index
    
    # Add columns
    df["weekNumber"] = df["ts"].apply(getWeekNumber)
    df["weekday"] = df["ts"].apply(getWeekday)
    df["timeRef"] = df["ts"].apply(getYearAndWeek)
    
    # Last processings...
    df = keepFullDays(df)
    df = df.sort_values(by='ts')
    
    return df

In [ ]:
# Test
df = formatDataFrameFromJSON(filename)
plt.figure()
df["open"].plot()

## Day - get Max&Min percentages 
Example with Week 9 : 27-02 -> 05-03

In [ ]:
# Example with week 9
try:
    df_week = df[df.weekNumber == 9]

    # Get monday, open & end
    df_day = df_week[df_week["weekday"] == 1]
    openDate = df_day.index[0].replace(hour=5)
    endDate = openDate.replace(hour=21)
    print("open at: ", openDate, ". End: ", endDate)

    open = df_day.loc[openDate]["open"]
    print("Open: ", open)

    # Get data in the time range [open-end]
    time_range = df_day[(df_day["ts"] < endDate) & (df_day["ts"] >= openDate)]
    max = time_range["high"].max()
    min = time_range["low"].min()
    print("Max: ", max)
    print("Min: ", min)

    # Get percentage
    percentageMax = (max - open)/open
    percentageMin = (min - open)/open

    print("Percentage max: ",(percentageMax*100).round(2), "%")
    print("Percentage min: ",(percentageMin*100).round(2), "%")
except:
    print("ERROR")

## Dataframe with percentage by hours

In [ ]:
def getPercentageByDay(df_day, params):
    high, low = getHighAndLowPercentageByDay(df_day, params)
    return high, low

def getHighAndLowPercentageByDay(df_day, params):
    #print(params)
    #print(df_day.head())
    openDate = df_day.index[0].replace(hour=params["beginHour"])
    open = df_day.loc[openDate]["open"]

    high_day_serie = pd.Series([])
    low_day_serie = pd.Series([])
    for i in range(params['beginHour'], params["endHour"] + 1):
        date = df_day.index[0].replace(hour=i)
        
        high = df_day.loc[date]["high"]
        low = df_day.loc[date]["low"]
        
        high_percentage = getPercentage(open, high)
        low_percentage = getPercentage(open, low)
        
        high_day_serie = high_day_serie.append(pd.Series([high_percentage]), ignore_index=True)
        low_day_serie = low_day_serie.append(pd.Series([low_percentage]), ignore_index=True)
    
    high_day_serie = high_day_serie.rename(df_day.index[0])
    low_day_serie = low_day_serie.rename(df_day.index[0])
    
    return high_day_serie, low_day_serie    

def getPercentage(open, high):
    percentage = (high - open)/open
    percentage = (percentage*100).round(3)
    return percentage


In [ ]:
### Test with Monday on Week 9
def __main__():
    df_week = df[df.weekNumber == 9]
    df_day = df_week[df_week["weekday"] == 1]
    params = {
        "beginHour" : 6,
        "endHour" : 18
    }
    df_p = pd.DataFrame()
    
    high, low = getPercentageByDay(df_day, params)
    df_p = df_p.append(high)
    df_p = df_p.append(low)
    return df_p

try:
    __main__().tail()
except:
    print("ERROR")

In [ ]:
def getPercentageByWeek(df_week, params):
    df_high_percentage = pd.DataFrame()
    df_low_percentage = pd.DataFrame()
    days = df_week.weekday.unique()
    for i in days:
        df_day = df_week[df_week["weekday"] == i]
        df_high_percentage_day, df_low_percentage_day = getPercentageByDay(df_day, params)
        
        df_high_percentage = df_high_percentage.append(df_high_percentage_day)
        df_low_percentage = df_low_percentage.append(df_low_percentage_day)
        
    df_high_percentage.columns = range(params["beginHour"], params["endHour"] + 1)
    df_low_percentage.columns = range(params["beginHour"], params["endHour"] + 1)
    
    dfs = {
        "highs" : df_high_percentage,
        "lows" : df_low_percentage
    }
    return dfs

In [ ]:
def __main__():
    df_week = df[df.weekNumber == 9]
    
    params = {
        "beginHour" : 5,
        "endHour" : 18
    }
    dfs = getPercentageByWeek(df_week, params)
    return dfs

try:
    d = __main__()
    d["lows"]
except:
    print("ERROR")

In [ ]:
def getHighLowDF(df, params):
    df_year_high = pd.DataFrame()
    df_year_low = pd.DataFrame()
    weeks = df.weekNumber.unique()
    for i in weeks:
        df_week = df[df.weekNumber == i]
        df_week_percentage = getPercentageByWeek(df_week, params)
        
        df_year_high = df_year_high.append(df_week_percentage["highs"])
        df_year_low = df_year_low.append(df_week_percentage["lows"])
    return {"highs": df_year_high, "lows": df_year_low}


In [ ]:
# Test
try:
    params = {
        "beginHour" : 6,
        "endHour" : 18
    }
    df_highs_lows = getHighLowDF(df, params)
    df_highs_lows["highs"].tail()
except:
    print("ERROR")

## Draw heatmaps

In [ ]:
try:
    plt.figure(figsize=(20,30))
    sns.heatmap(df_highs_lows["lows"], vmin=-8, vmax=0)
except:
    print("ERROR")

---
# Part 2. Process gain 

In [ ]:
def isRowWinning(df, index, params):
    for i in range(params["beginHour"], params["endHour"] + 1):
        high = df["highs"].loc[index][i]
        low  = df["lows"].loc[index][i]
        if(high > params["high_rate"]):
            return 1
        elif(low < params["low_rate"]):
            return -1
    return 0

In [ ]:
# Test
try:
    params = {
        "high_rate" : 1.4,
        "low_rate" : -8,
        "fees_rate" : 0.25/100,
        "beginHour" : 6,
        "endHour" : 18
    }
    nbWins = 0
    nbLoses = 0

    df_wins = pd.DataFrame()
    for index, row in df_highs_lows["highs"].iterrows():
        res = isRowWinning(df_highs_lows, index, params)
        open = df.loc[index]["open"]
        high = df_highs_lows["highs"].loc[index].max()
        low = df_highs_lows["lows"].loc[index].min()

        serie = pd.Series([res, open, high, low], index=["win", "open", "high", "low"])
        serie = serie.rename(index)
        df_wins = df_wins.append(serie)

    nbDays = df_highs_lows["highs"].shape[0]
    missing = nbDays - (nbWins+nbLoses)
    print(nbWins, nbLoses, nbWins + nbLoses,  missing)        
    print(df_wins.head())
    plt.figure(figsize=(15,15))
    df_wins.loc[:"2017-02-01", "win"].plot()
except:
    print("ERROR")

In [ ]:
def getWins(df, params): 
    df_wins = pd.DataFrame()
    for index, row in df["highs"].iterrows():
        res = isRowWinning(df, index, params)
        high = df["highs"].loc[index].max()
        low = df["lows"].loc[index].min()

        serie = pd.Series([res, high, low], index=["win", "high", "low"])
        serie = serie.rename(index)
        df_wins = df_wins.append(serie)
    return df_wins


In [ ]:
## Test
try: 
    params = {
        "high_rate" : 1.5,
        "low_rate" : -8,
        "fees_rate" : 0.25/100,
        "beginHour" : 6,
        "endHour" : 18
    }
    nbWins = 0
    nbLoses = 0

    df_wins = getWins(df_highs_lows, params)

    nbDays = df_highs_lows["highs"].shape[0]
    missing = nbDays - (nbWins+nbLoses)
    print(nbWins, nbLoses, nbWins + nbLoses,  missing)        
    print(df_wins.head())
    plt.figure(figsize=(15,15))

    df_wins.loc["2017-11-01":, "win"].plot()
except:
    print("ERROR")

In [ ]:
def getAmountAfterTransaction(amount, fees_rate, rate):
    amount_after_buying = amount * (1 - fees_rate)
    amount_before_selling = amount_after_buying * (1 + rate/100)
    amount_after_selling = amount_before_selling * (1 - fees_rate)
    diff = amount_after_selling - amount
    return diff

In [ ]:
def getGain(df_wins, amount, params):
    gain = 0
    for index,row in df_wins.iterrows():
        win = row["win"]
        rate = params["high_rate"]
        if(win < 1):
            rate = params["low_rate"]
        gain = gain + getAmountAfterTransaction(amount, params["fees_rate"], rate)
    return gain

In [ ]:
try:
    df.head()
    df.loc["2017-12":]
except:
    print("ERROR")

In [ ]:
# Test
try:
    amount = 1000
    params = {
        "high_rate" : 1.9,
        "low_rate" : -4,
        "fees_rate" : 0.25/100,
        "beginHour" : 6,
        "endHour" : 19
    }

    df_last_semester = df["2017-12":]
    df_highs_lows = getHighLowDF(df_last_semester, params)
    df_highs_lows["highs"].head()
    df_wins = getWins(df_highs_lows, params)
    getGain(df_wins, amount, params)
except:
    print("ERROR")

## Hyper tuning

In [ ]:
def hyper_tuning(df, beginHours, endHours, low_rates, high_rates, fees_rate, amount):
    df_gains = pd.DataFrame()

    nb_configurations = len(low_rates) * len(high_rates) * len(beginHours)
    remaining_loops = nb_configurations
    print("Running", nb_configurations, "configurations.")

    for beginHour in beginHours:
        for low_rate in low_rates:
            for high_rate in high_rates:
                for endHour in endHours:
                    params = {
                        "beginHour": beginHour,
                        "endHour": endHour,
                        "low_rate": low_rate,
                        "high_rate": high_rate,
                        "fees_rate": fees_rate
                    }
                    df_highs_lows = getHighLowDF(df, params)
                    df_wins = getWins(df_highs_lows, params)
                    gain = getGain(df_wins, amount, params)

                    index = str(params["low_rate"]) + "%_" \
                        + str(params["high_rate"]) + "%__" \
                        + str(params["beginHour"]) + "h-" \
                        + str(params["endHour"]) + "h"
                    serie = pd.Series(
                        [gain, params["low_rate"], params["high_rate"], \
                             params["beginHour"], params["endHour"], index], \
                        index=["gain", "low_rate", "high_rate", "beginHour", "endHour", "ticks"])
                    serie = serie.rename(index)

                    df_gains = df_gains.append(serie)
                    remaining_loops = remaining_loops - 1
                    print(index, gain,"$", " (", format(100 - remaining_loops*100/nb_configurations, ".1f"),"%)")
                    
    return df_gains


In [ ]:
# Test
def __main__():
    return
    try:
        low_rates = range(-2, -6, -1)
        high_rates = np.arange(1.0, 3.0, 0.1)
        beginHours = range(2,10,1)
        endHours = range(18,19,1)

        fees_rate = 0.24/100
        amount = 1000

        df_gains = hyper_tuning(df["2018-01-08":"2018-01-14"], beginHours, endHours, low_rates, high_rates, fees_rate, amount)
    except:
        print("ERROR")
        
__main__()

### Plotting

In [ ]:
# Show best configurations
df_gains[df_gains["gain"] > 0].sort_values(by='gain', ascending=False)["gain"]

In [ ]:
# Show when winning
plt.figure(figsize=(20,10))
plot = df_gains["gain"].plot(rot=45)


## Explore with all year data

In [ ]:
def getGainFromDF(df, amount, params):
    df_highs_lows = getHighLowDF(df, params)
    df_wins = getWins(df_highs_lows, params)
    gain = getGain(df_wins, amount, params)
    return gain

In [ ]:
def getWinsFromDF(df, params):
    df_highs_lows = getHighLowDF(df, params)
    df_wins = getWins(df_highs_lows, params)
    return df_wins

In [ ]:
amount = 1000
params = {
    "high_rate": 5,
    "low_rate": -5,
    "beginHour":5,
    "endHour":  15,
    "fees_rate": 0.24/100
}

df_m1 = df[ "2018-01" ]
df_w2 = df[ df["weekNumber"] == 2 ]
df_w3 = df[ df["weekNumber"] == 3 ]
df_w4 = df[ df["weekNumber"] == 4 ]

df_week = df_m1

# Wins
df_wins = getWinsFromDF(df_week, params);
plt.figure(figsize=(20,5))
df_wins["win"].plot()

# Gain
gain = getGainFromDF(df_week, amount, params)
print("Gain {gain}$ ".format(gain=gain))

In [ ]:
# Heatmaps
df_highs_lows = getHighLowDF(df_week, params)

plt.figure(figsize=(20,5))
sns.heatmap(df_highs_lows["highs"], vmin=0, vmax=2, annot=True)
plt.figure(figsize=(20,5))
sns.heatmap(df_highs_lows["lows"], vmin=-7, vmax=0, annot=True)

In [ ]:
def getOpensByDay(df):
    daily_df = pd.DataFrame()
    for i in df["weekday"].unique():
        current_series = df[ df["weekday"] == i]["open"]
        day = current_series.index[0].strftime("%A")
        daily_df[day] = current_series.values
    return daily_df

In [ ]:
plt.figure(figsize=(20,8))
getOpensByDay(df_w3).plot()
#plt.figure(figsize=(20,5))

#getOpensByDay(df_w3).plot()
#getOpensByDay(df_w4).plot()
